In [18]:
import requests
import geojson
import aiohttp
import asyncio
from geocodio import GeocodioClient

import pandas as pd
import matplotlib as plt
import rasterio
from rasterio import features
import numpy as np

from dotenv import load_dotenv
import os

CALCULATING CENTER FOR EACH CLUSTER

In [19]:
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import cascaded_union

def get_polygons_center(clustered_tif_path):
    with rasterio.open(clustered_tif_path) as src:
        data = src.read()
        image = src.read(1)
        global clusters
        clusters = src.read(2)

    class_vals = set(clusters.flatten()) - {-1}
    centers = []
    pcords = []
    manual_centers = []
    for class_val in class_vals:
        mask = clusters == class_val
        shapes = features.shapes(mask.astype('uint8'), transform=src.transform)
        local_mc = []
        polygons = []
        for polygon, value in shapes:
            polygons.append(Polygon(polygon['coordinates'][0]))
            mc = [0,0]
            mc[1] = sum([i[0] for i in polygon['coordinates'][0]])/len(polygon['coordinates'][0])
            mc[0] = sum([i[1] for i in polygon['coordinates'][0]])/len(polygon['coordinates'][0])
            local_mc.append(tuple(mc))
            pcords.append(polygon['coordinates'][0])
        manual_centers.append(tuple(sum(coords)/len(coords) for coords in zip(*local_mc)))
        multipolygon = MultiPolygon(polygons)
        center_pixel = multipolygon.centroid
        center_realworld = rasterio.transform.xy(src.transform, center_pixel.x, center_pixel.y)
        centers.append((center_realworld[1], center_realworld[0]))
    return manual_centers


In [20]:
centers = get_polygons_center("C:/Users/User/PycharmProjects/SafeRoute/clustered.tif")# TODO use relative path
len(centers)

247

In [21]:
min(centers)

(40.4233495399713, -74.17441381166898)

In [22]:
max(centers)

(41.38895668460448, -73.68139153365186)

In [23]:
load_dotenv()

True

REQUESTING DATA FROM GEOCODIO

In [24]:
gr = GeocodioClient(os.getenv("GEOCODE_API_KEY"))
address_sets = gr.reverse(centers, fields = ["acs-economics"])

In [25]:
address_sets[0]

{'results': [{'address_components': {'number': '32',
    'street': 'State Rte 22',
    'formatted_street': 'State Rte 22',
    'city': 'Valhalla',
    'county': 'Westchester County',
    'state': 'NY',
    'zip': '10595',
    'country': 'US'},
   'formatted_address': '32 State Rte 22, Valhalla, NY 10595',
   'location': {'lat': 41.089896, 'lng': -73.754399},
   'accuracy': 0.99,
   'accuracy_type': 'rooftop',
   'source': 'Westchester',
   'fields': {'census': {'2021': {'census_year': 2021,
      'state_fips': '36',
      'county_fips': '36119',
      'tract_code': '012301',
      'block_code': '2000',
      'block_group': '2',
      'full_fips': '361190123012000',
      'place': None,
      'metro_micro_statistical_area': {'name': 'New York-Newark-Jersey City, NY-NJ-PA',
       'area_code': '35620',
       'type': 'metropolitan'},
      'combined_statistical_area': {'name': 'New York-Newark, NY-NJ-CT-PA',
       'area_code': '408'},
      'metropolitan_division': {'name': 'New York-Je

In [26]:
household_income = address_sets[2]['results'][0]['fields']['acs']['economics']['Median household income']['Total']['value']
household_income

91250

In [27]:
print([address_sets[0]['results'][i]['location']['lat'] for i in range(len(address_sets[0]['results']))])
print([address_sets[0]['results'][i]['location']['lng'] for i in range(len(address_sets[0]['results']))])
print(centers[0])

[41.089896, 41.097692, 41.097487, 41.09731, 41.098093, 41.098093, 41.095577, 41.094194, 41.092902]
[-73.754399, -73.760441, -73.761207, -73.762283, -73.760421, -73.760421, -73.765901, -73.767164, -73.768307]
(41.08957727110209, -73.75602149612448)


In [29]:
cords = []
for result in address_sets:
    if result['results']:
        # check is for keyerror due to bad data.(value if exists else 0)
        income_list = [result['results'][i]['fields']['acs']['economics']['Median household income']['Total']['value'] 
                       if result['results'][i]['fields']['acs']['economics']['Median household income'].get('Total', 0) 
                       else 0 for i in range(len(result['results']))]
        income = sum(list(filter(lambda x: x != 0, income_list)))/len(income_list)
        cords.append(income)

GETTING CLUSTER SCORE FOR EACH CLUSTER FROM REQUESTED DATA

In [31]:
cluster_score = {c: 0 for c in np.unique(clusters)}
for i in range(0, len(cluster_score)-1):
    cluster_score[i] = int(cords[i])
cluster_score

{-1: 0,
 0: 146636,
 1: 167841,
 2: 91250,
 3: 128688,
 4: 119538,
 5: 152981,
 6: 202500,
 7: 161670,
 8: 240500,
 9: 168012,
 10: 136507,
 11: 99308,
 12: 194266,
 13: 135385,
 14: 77829,
 15: 58906,
 16: 25738,
 17: 130666,
 18: 190781,
 19: 84952,
 20: 74496,
 21: 78243,
 22: 0,
 23: 23038,
 24: 53701,
 25: 0,
 26: 53701,
 27: 0,
 28: 0,
 29: 72566,
 30: 85833,
 31: 0,
 32: 41968,
 33: 87788,
 34: 116932,
 35: 77660,
 36: 0,
 37: 0,
 38: 3431,
 39: 0,
 40: 67798,
 41: 47218,
 42: 85841,
 43: 92315,
 44: 128571,
 45: 0,
 46: 66674,
 47: 0,
 48: 136214,
 49: 77038,
 50: 114063,
 51: 23319,
 52: 47569,
 53: 45378,
 54: 41537,
 55: 250001,
 56: 36552,
 57: 47019,
 58: 77072,
 59: 93611,
 60: 53900,
 61: 173750,
 62: 46915,
 63: 46577,
 64: 114076,
 65: 54743,
 66: 96771,
 67: 66250,
 68: 87604,
 69: 116435,
 70: 211250,
 71: 144031,
 72: 74080,
 73: 76139,
 74: 136237,
 75: 103781,
 76: 104524,
 77: 0,
 78: 186250,
 79: 250001,
 80: 250001,
 81: 142213,
 82: 96875,
 83: 96875,
 84: 174

SAVING NORMAL DATAFRAME

In [32]:
df = pd.DataFrame(list(cluster_score.items()), columns=["cluster", "income"])
df.to_csv('average_income.csv', index=False)
df

,cluster,income
0,-1,0
1,0,146636
2,1,167841
3,2,91250
4,3,128688
...,...,...
243,242,178750
244,243,91250
245,244,98036
246,245,139865
